# Caradisiac Timeline Analysis

# Cmd to extract some tweets
#### python Exporter.py --username "caradisiac" --since 2010-01-01 --until 2018-03-12 --maxtweets 10000 --output caradisiac.csv

#### python Exporter.py --username "turbofr" --since 2010-01-01 --until 2018-03-12 --maxtweets 10000 --output turbofr.csv

#### python Exporter.py --username "BlogNewsAuto" --since 2010-01-01 --until 2018-03-12 --maxtweets 10000 --output blognewsauto.csv

#### python Exporter.py --username "AUTOMOTIVPRESS" --since 2010-01-01 --until 2018-03-12 --maxtweets 10000 --output automotivpress.csv

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

import numpy as np
import json
import pandas as pd

import sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

import nltk
from nltk.stem.snowball import FrenchStemmer
from nltk.util import ngrams
from nltk.corpus import stopwords
from nltk.stem.porter import *
import string
import re
from unidecode import unidecode
punctuation = set(string.punctuation)


# Reading tweets

In [2]:
tweet_repository = '../GetOldTweets-python-master/'
tweet_files = ["caradisiac.csv", "turbofr.csv", "blognewsauto.csv", "automotivpress.csv"]

df = pd.DataFrame()
list_ = []
for file_ in tweet_files:
    frame = pd.read_csv(tweet_repository+file_,index_col=None, header=0, sep="|")
    list_.append(frame)
df = pd.concat(list_)

In [3]:
df = df[["date", "retweets", "favorites", "text", "hashtags"]]
df.sample(3)

,date,retweets,favorites,text,hashtags
11,2016-10-02 18:43,1,6,Le meilleur cabriolet du #MondialAuto http:// ...,#MondialAuto #MondialAuto
3984,2017-03-28 14:05,5,8,#Diapo : La nouvelle Alpine A470 engagée dans ...,#Diapo
9914,2011-09-30 11:42,0,0,Kubica : réponse début novembre http:// bit.ly...,NaN


In [4]:
df.shape

(31040, 5)

# Cleaning function

In [5]:
def remove_accents(input_str):
    '''
    Entrée : input_str au format unicode
    Enlève les accents pour un unicode
    '''
    return unidecode(input_str);  # remove accent character


# Définition des stopwords
french_stopwords = set(stopwords.words('french'))

# Le stemming que nous utilisons
stemmer = FrenchStemmer()

In [6]:
def clean_function(x, stemming = True):
    '''
    Fonction de traitement de texte
    Entrée : Une chaîne de caractères
    
    -> Décode en UTF8
    -> lower case
    -> Enlève la ponctuation
    -> Enlève les stopwords
    -> Enlève les accents
    -> Traduit les abréviations courantes
    -> Utilise le stemming (met le mot à sa racine), algo snowball
    '''
    # Si le texte n'est pas vide : clean.
    # text = x.encode("UTF8")

    # Lower case
    doc = "".join([w for w in x.lower()])

    #Punctuation removal
    regex = re.compile("[%s]" % re.escape(string.punctuation))
    doc = regex.sub(" ", doc)

    # Remove accents
    doc = [remove_accents(w) for w in doc.split()]

    # Remove stopwords
    doc = [w for w in doc if (w not in french_stopwords) & (len(w) > 1)]
    # doc = [w for w in doc.split()]

    # Stemming
    if stemming:
        doc = [stemmer.stem(w) for w in doc]
    else:
        pass

    doc = " ".join(w for w in doc)
    return doc

In [16]:
%%time
df['cleaned_text'] = df['text'].apply(clean_function)
df['cleaned_text_no_stem'] = df['text'].apply(lambda x: clean_function(x, False))

CPU times: user 14.6 s, sys: 44.4 ms, total: 14.7 s
Wall time: 14.7 s


In [17]:
df

,date,retweets,favorites,text,hashtags,cleaned_text,cleaned_text_no_stem
0,2018-03-09 10:28,0,0,Les affaires se succèdent dans le dossier du c...,#xtor,le affair succedent dossi control station pari...,les affaires succedent dossier controle statio...
1,2018-03-08 10:25,1,0,Un responsable de #Volkswagen l'a confirmé au ...,#Volkswagen #xtor,respons volkswagen confirm salon genev troisie...,responsable volkswagen confirme salon geneve t...
2,2018-03-07 09:54,2,4,#Alpine vient de dévoiler au salon les finitio...,#Alpine #xtor,alpin vient devoil salon le finit classiqu nou...,alpine vient devoiler salon les finitions clas...
3,2018-03-06 11:50,1,3,"Comme chaque année, l'ouverture du salon de Ge...",#xtor,comm chaqu anne ouvertur salon genev revel voi...,comme chaque annee ouverture salon geneve reve...
4,2018-03-05 10:25,5,2,De plus en plus de marques renoncent à mettre ...,#xtor,plus plus marqu renoncent mettr mannequin pre ...,plus plus marques renoncent mettre mannequins ...
5,2018-03-02 10:52,3,6,"Et vous, quelle boîte vous préférez ? http://w...",#xtor,quel boit pref http www caradisiac com voitur ...,quelle boite preferez http www caradisiac com ...
6,2018-03-01 17:37,3,2,"Essai longue durée en @Dacia_FR , l'équipe Car...",#xtor,essai longu dure daci fr equip caradisiac effe...,essai longue duree dacia fr equipe caradisiac ...
7,2018-02-27 10:05,3,2,"Selon le Financial Times, Fiat-Chrysler ne ven...",#xtor,selon financial tim fiat chrysl vendr plus voi...,selon financial times fiat chrysler vendra plu...
8,2018-02-26 10:27,3,4,"Avant le printemps, nous aurons droit au Salon...",#xtor,avant printemp droit salon genev rassembl touj...,avant printemps droit salon geneve rassemble t...
9,2018-02-23 10:26,3,4,La Préfecture de Police montre l'exemple en ma...,#Golf #Passat #xtor,prefectur polic montr exempl matier transit ec...,prefecture police montre exemple matiere trans...


In [13]:
df.to_csv("../output/cleaned_data.csv", sep=";", index=False)

In [18]:
df["cleaned_text"].to_csv("../output/cleaned_tweets.csv", sep=";", index=False)
df["cleaned_text_no_stem"].to_csv("../output/cleaned_tweets_no_stem.csv", sep=";", index=False)

# TF-IDF

In [19]:
%%time
vectorizer = CountVectorizer(ngram_range=(1,2))
wordCount = vectorizer.fit_transform(df['cleaned_text'].values)

tfidf = TfidfTransformer(smooth_idf=True, sublinear_tf = True).fit_transform(wordCount)

CPU times: user 1.53 s, sys: 69.9 ms, total: 1.6 s
Wall time: 1.62 s


In [20]:
tfidf

<31040x223157 sparse matrix of type '<class 'numpy.float64'>'
	with 925280 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.cluster import KMeans, k_means

In [23]:
%%time
clusters = k_means(X = tfidf, n_clusters = 20)
labels = clusters[1]
clustered = pd.DataFrame()
clustered['word'] = words
clustered['label'] = labels
clustered.groupby(labels).count()

NameError: name 'words' is not defined